# Test your model
Make sure you are in the object detection folder


In [0]:
from os.path import join
from google.colab import drive
 
ROOT = "/content/drive"
drive.mount(ROOT)

# This is a custom path. 'My Drive' is the root of your google drive change accordingly
PROJ = "My Drive/workshop/models/research/object_detection" 
PROJECT_PATH = join(ROOT, PROJ)
 
#%cd ~/content 
%cd {PROJECT_PATH}



In [0]:
import os
cwd = os.getcwd()
print ("The current directory is:"+cwd)

In [0]:
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

# Name of the directory containing the object detection model that we created
MODEL_NAME = 'inference_graph'
images_path = ROOT + "/My Drive/workshop/models/research/object_detection/images/validation"
IMAGE_NAME = images_path+"/validation1.JPG"


# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

# Path to image
PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)

# Number of classes the object detector can identify
NUM_CLASSES = 6

In [0]:
# Load the label map.
# Label maps map indices to category names, so that when our convolution
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

In [0]:
# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
image = cv2.imread(PATH_TO_IMAGE)
image_expanded = np.expand_dims(image, axis=0)



In [0]:
# Perform the actual detection by running the model with the image as input
(boxes, scores, classes, num) = sess.run([detection_boxes, detection_scores, detection_classes, num_detections],feed_dict={image_tensor: image_expanded})



In [0]:
# Draw the results of the detection (aka 'visulaize the results')

from google.colab.patches import cv2_imshow

min_score_thresh_ = 0.80
vis_util.visualize_boxes_and_labels_on_image_array(
    image,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=5,
    min_score_thresh = min_score_thresh_)

# All the results have been drawn on image. Now display the image.
print("==========Detection==========")
cv2_imshow( image)
# This is the way I'm getting my coordinates
boxes_ = np.squeeze(boxes)
max_boxes_to_draw = boxes_.shape[0]
scores_ = np.squeeze(scores)
classes_ = np.squeeze(classes).astype(np.int32) 
for i in range(min(max_boxes_to_draw, boxes_.shape[0])):
    if scores_ is None or scores_[i] > min_score_thresh_:
        # boxes[i] is the box which will be drawn
        print ("Detected cell: class ", list(filter(lambda detected: detected['id'] == classes_[i], categories)), " score ", scores_[i]  )
        